In [107]:
using Plots
using LinearAlgebra
using NamedArrays

Plots.gr();

In [101]:
# Basic terrain cost field for initial pathing tests

basicTerrain(x, y) = exp(-1(x^2 + y^2))

# Interpret as going from -boundary to boundary in resolution steps
boundary = 1
resolution = 2

# Generate discrete field
discreteField = zeros(boundary * resolution * 2 + 1, boundary * resolution * 2 + 1)

for i in 1:resolution*boundary*2+1
  for j in 1:resolution*boundary*2+1
    
    # Rearange axis
    x = (i - resolution * boundary - 1) / resolution
    y = (j - resolution * boundary - 1) / resolution
    discreteField[i,j] = round(basicTerrain(x,y), digits=3)
  
  end
end

# Show graph
show(stdout, "text/plain", discreteField)

f(x, y) = sin(x) + cos(y)
#surface(-boundary:0.1/resolution:boundary, -boundary:0.1/resolution:boundary, basicTerrain)

5×5 Matrix{Float64}:
 0.135  0.287  0.368  0.287  0.135
 0.287  0.607  0.779  0.607  0.287
 0.368  0.779  1.0    0.779  0.368
 0.287  0.607  0.779  0.607  0.287
 0.135  0.287  0.368  0.287  0.135

f (generic function with 1 method)

In [112]:
# Text discrete field matrix
discreteField = [1 4 7; 
                 2 5 8; 
                 3 6 9]

# Conection matrix from discrete field
connectionMatrix = fill(Inf, (length(discreteField), length(discreteField)))

# Dimensions from discrete field are usefull
verticalSize = size(discreteField, 1)
horizontalSize = size(discreteField, 2)

#println("verticalSize ", verticalSize)
#println("horizontalSize ", horizontalSize)

# Complete conection matrix
# Algorithm has complexity O(n²)
for j in axes(discreteField, 2)
  for i in axes(discreteField, 1)
    # Matrix is iterated from top to bottom
    # then from left to right
    loc = (j - 1) * verticalSize + i
    #println((i,j)," ", loc)

    # Not at left boundary
    if loc > verticalSize
      #println("Left connection at ", (i,j), " to ", (i,j - 1))
      connectionMatrix[loc,loc - horizontalSize] = abs(discreteField[i,j] - discreteField[i,j - 1])
    end

    # Not at right boundary
    if loc ≤ verticalSize * (horizontalSize - 1)
      #println("Right connection at ", (i,j), " to ", (i,j + 1))
      connectionMatrix[loc,loc + horizontalSize] = abs(discreteField[i,j] - discreteField[i,j + 1])
    end
    
    # Not at bottom boundary
    if loc % verticalSize != 0
      #println("Lower connection at ", (i,j), " to ", (i + 1,j))
      connectionMatrix[loc,loc + 1] = abs(discreteField[i,j] - discreteField[i + 1,j])
    end

    # Not at top boundary
    if loc % verticalSize != 1
      #println("Upper connection at ", (i,j), " to ", (i - 1,j))
      connectionMatrix[loc,loc - 1] = abs(discreteField[i,j] - discreteField[i - 1,j])
    end
  end
end

# Visually clear matrix
namedArray = NamedArray(connectionMatrix)
setdimnames!(namedArray, ("Origin", "Destiny"));
namedArray

9×9 Named Matrix{Float64}
Origin ╲ Destiny │   1    2    3    4    5    6    7    8    9
─────────────────┼────────────────────────────────────────────
1                │ Inf  1.0  Inf  3.0  Inf  Inf  Inf  Inf  Inf
2                │ 1.0  Inf  1.0  Inf  3.0  Inf  Inf  Inf  Inf
3                │ Inf  1.0  Inf  Inf  Inf  3.0  Inf  Inf  Inf
4                │ 3.0  Inf  Inf  Inf  1.0  Inf  3.0  Inf  Inf
5                │ Inf  3.0  Inf  1.0  Inf  1.0  Inf  3.0  Inf
6                │ Inf  Inf  3.0  Inf  1.0  Inf  Inf  Inf  3.0
7                │ Inf  Inf  Inf  3.0  Inf  Inf  Inf  1.0  Inf
8                │ Inf  Inf  Inf  Inf  3.0  Inf  1.0  Inf  1.0
9                │ Inf  Inf  Inf  Inf  Inf  3.0  Inf  1.0  Inf